In [1]:
import pandas as pd
import numpy as np
stations=pd.read_csv("/kaggle/input/carlocations/stations.csv")
vehicles=pd.read_csv("/kaggle/input/carlocations/vehicles.csv")
import random
import math
from deap import base, creator, tools
stations["cost"]=stations["cost"]

In [2]:
randomm =[np.random.rand() for i in range(stations["cost"].shape[0])]
stations["cost"]=stations["cost"]+randomm

In [3]:


# Define the problem parameters
car_locations= vehicles.values.copy()
station_locations= stations.values.copy()


# SPEA2

In [4]:

# Define the chromosome structure
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

# Define the genetic operators
def evaluate(individual):
    
    total_distance = 0
    total_cost = 0
    station_count = [0] * len(station_locations)
    for car_index, station_index in enumerate(individual):
        car_x, car_y = car_locations[car_index]
        station_x, station_y, station_price = station_locations[station_index]
        distance = math.sqrt((car_x - station_x) ** 2 + (car_y - station_y) ** 2)
        total_distance += distance
        total_cost += station_price
        station_count[station_index] += 1
    # Add constraint for maximum number of visits to a station
    for count in station_count:
        if count > 6:
            return total_distance+count,  total_cost+count
    
    
    return total_distance, total_cost

def random_station():
    return random.randint(0, len(station_locations) - 1)

def mutation(individual):
    car_index = random.randint(0, len(individual) - 1)
    individual[car_index] = random_station()
    return individual,None

def crossover(parent1, parent2):
    cxpoint = random.randint(1, len(parent1) - 1)
    child1 = parent1[:cxpoint] + parent2[cxpoint:]
    child2 = parent2[:cxpoint] + parent1[cxpoint:]
    return creator.Individual(child1), creator.Individual(child2)

toolbox.register("evaluate", evaluate)
toolbox.register("individual", tools.initRepeat, creator.Individual, random_station, len(car_locations))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selSPEA2)#SPEA2

# Define the main function
def main():
    population_size = 100
    num_generations = 100
    cxpb = 0.5
    mutpb = 0.2
    # Create the initial population
    population = toolbox.population(n=population_size)
    
    
    # Evaluate the fitness of each individual in the population
    fitnesses = [toolbox.evaluate(individual) for individual in population]
    
    for individual, fitness in zip(population, fitnesses):  
        individual.fitness.values = fitness

    # Run the evolution
    for generation in range(num_generations):
        # Select the parents for the next generation
        
        offspring = toolbox.select(population,20# len(population)
                                  )

        # Apply crossover to create the children
        offspring = [toolbox.mate(parent1, parent2) for parent1, parent2 in zip(offspring[::2], offspring[1::2])]
        offspring = [child for pair in offspring for child in pair]

        # Apply mutation to create new individuals
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)

        # Evaluate the fitness of the new individuals
        #print(offspring)
        
        fitnesses = [toolbox.evaluate(individual) for individual in offspring]
        for individual, fitness in zip(offspring, fitnesses):
            individual.fitness.values = fitness

        # Combine the parents and children to create the next generation
        population[:] = toolbox.select(population + offspring, population_size)
        
        
        # Print the best individuals in the current generation
        best_individuals = tools.selBest(population, k=10)
        #print("Generation", generation)
        #for individual in best_individuals:
            #print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
        # extract Pareto front
        pareto_front = tools.sortNondominated(population, k=10, first_front_only=True)[0]
    print("Pareto Front de SPEA2:")
    unique_front=[]
    for individual in pareto_front:
        if individual not in unique_front:
            print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
            print("-----------------------------------------------------")
            unique_front.append(individual)
    #return offspring


main()

Pareto Front de SPEA2:
  Distance: 403.4474536759368 Cost: 75.58210889707959 Assignments: [92, 98, 100, 98, 45, 88, 65, 60, 24, 5, 4, 36, 52, 89, 1, 100, 28, 66, 14, 36, 60, 5, 86, 100, 28, 79, 36, 36, 11, 53, 65, 22, 55, 83, 48, 68, 24, 1, 89, 51, 90, 101, 22, 69, 3, 60, 3, 15, 66, 83, 3, 47, 54, 99, 6, 1, 20, 44, 60, 83, 15, 86, 90, 54, 90, 89, 65, 56, 55, 90, 76, 20, 43, 16, 66, 66, 69, 101, 54, 60, 82, 15, 60, 55, 37, 57, 55, 24, 45, 55, 37, 90, 64, 19, 99, 97, 100, 89, 11, 36, 6, 24]
-----------------------------------------------------
  Distance: 415.7350628829682 Cost: 73.54799947207083 Assignments: [92, 98, 100, 32, 45, 69, 65, 60, 24, 5, 4, 36, 52, 89, 1, 100, 28, 66, 14, 36, 60, 5, 86, 100, 28, 79, 36, 36, 11, 92, 65, 22, 55, 83, 48, 68, 24, 1, 89, 51, 90, 101, 22, 69, 3, 60, 3, 15, 66, 83, 3, 47, 54, 99, 6, 1, 20, 44, 60, 83, 15, 86, 90, 54, 90, 89, 65, 56, 55, 90, 76, 20, 65, 16, 66, 66, 69, 101, 54, 60, 82, 15, 60, 55, 37, 57, 55, 24, 45, 55, 37, 90, 64, 19, 99, 97, 100, 

# NSGA2

In [5]:
# Define the chromosome structure
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

# Define the genetic operators
def evaluate(individual):
    
    total_distance = 0
    total_cost = 0
    station_count = [0] * len(station_locations)
    for car_index, station_index in enumerate(individual):
        car_x, car_y = car_locations[car_index]
        station_x, station_y, station_price = station_locations[station_index]
        distance = math.sqrt((car_x - station_x) ** 2 + (car_y - station_y) ** 2)
        total_distance += distance
        total_cost += station_price
        station_count[station_index] += 1
    # Add constraint for maximum number of visits to a station
    for count in station_count:
        if count > 6:
            return total_distance+count,  total_cost+count
    
    
    return total_distance, total_cost

def random_station():
    return random.randint(0, len(station_locations) - 1)

def mutation(individual):
    car_index = random.randint(0, len(individual) - 1)
    individual[car_index] = random_station()
    return individual,None

def crossover(parent1, parent2):
    cxpoint = random.randint(1, len(parent1) - 1)
    child1 = parent1[:cxpoint] + parent2[cxpoint:]
    child2 = parent2[:cxpoint] + parent1[cxpoint:]
    return creator.Individual(child1), creator.Individual(child2)

toolbox.register("evaluate", evaluate)
toolbox.register("individual", tools.initRepeat, creator.Individual, random_station, len(car_locations))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selNSGA2)

# Define the main function
def main():
    population_size = 100
    num_generations = 100
    cxpb = 0.5
    mutpb = 0.2
    # Create the initial population
    population = toolbox.population(n=population_size)
    
    
    # Evaluate the fitness of each individual in the population
    fitnesses = [toolbox.evaluate(individual) for individual in population]
    
    for individual, fitness in zip(population, fitnesses):  
        individual.fitness.values = fitness

    # Run the evolution
    for generation in range(num_generations):
        # Select the parents for the next generation
        
        offspring = toolbox.select(population,20# len(population)
                                  )

        # Apply crossover to create the children
        offspring = [toolbox.mate(parent1, parent2) for parent1, parent2 in zip(offspring[::2], offspring[1::2])]
        offspring = [child for pair in offspring for child in pair]

        # Apply mutation to create new individuals
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)

        # Evaluate the fitness of the new individuals
        #print(offspring)
        
        fitnesses = [toolbox.evaluate(individual) for individual in offspring]
        for individual, fitness in zip(offspring, fitnesses):
            individual.fitness.values = fitness

        # Combine the parents and children to create the next generation
        population[:] = toolbox.select(population + offspring, population_size)
        
        
        # Print the best individuals in the current generation
        #best_individuals = tools.selBest(population, k=10)
        #print("Generation", generation)
        #for individual in best_individuals:
            #print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
        # extract Pareto front
        pareto_front = tools.sortNondominated(population, k=10, first_front_only=True)[0]
    print("Pareto Front with NSGA2:")
    unique_front=[]
    for individual in pareto_front:
        if individual not in unique_front:
            print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
            print("-----------------------------------------------------")
            unique_front.append(individual)
    return offspring

offspring=main()

/opt/conda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/opt/conda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Pareto Front with NSGA2:
  Distance: 385.0258109718852 Cost: 78.46503612921374 Assignments: [48, 83, 65, 51, 21, 46, 95, 101, 83, 5, 40, 99, 34, 100, 42, 37, 26, 65, 64, 97, 19, 99, 72, 79, 66, 50, 36, 15, 20, 52, 22, 100, 62, 27, 11, 99, 100, 95, 39, 98, 26, 99, 89, 65, 63, 46, 46, 101, 2, 68, 3, 44, 99, 57, 45, 55, 20, 32, 10, 35, 55, 10, 39, 4, 9, 1, 46, 63, 0, 72, 34, 50, 19, 7, 42, 1, 100, 9, 89, 43, 2, 89, 6, 99, 44, 57, 47, 47, 55, 65, 91, 45, 90, 32, 11, 65, 22, 60, 96, 55, 5, 79]
-----------------------------------------------------
  Distance: 381.9803086392677 Cost: 78.85744589221213 Assignments: [48, 83, 65, 51, 21, 46, 95, 101, 83, 5, 40, 99, 34, 100, 42, 37, 26, 65, 64, 97, 19, 99, 72, 79, 66, 50, 36, 15, 20, 52, 22, 100, 62, 27, 11, 99, 100, 95, 39, 98, 26, 99, 89, 65, 63, 46, 46, 14, 2, 68, 3, 44, 99, 57, 45, 55, 20, 32, 10, 35, 55, 10, 39, 4, 9, 1, 46, 63, 0, 72, 34, 50, 19, 7, 42, 1, 100, 9, 89, 43, 2, 89, 6, 99, 44, 57, 47, 47, 55, 65, 91, 45, 90, 32, 11, 60, 46, 60,

## Extra / not important 

In [6]:
import random
import math
from deap import base, creator, tools

# Define the problem parameters
car_locations = [[0, 0], [1, 1], [2, 2],[66,32]]
station_locations = [[3, 3, 10], [4, 4, 5], [5, 5, 8],[0, 0, 9],[3, 0, 7],
                    [10, 10, 97],[10, 10, 0]]

# Define the chromosome structure
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

# Define the genetic operators
def evaluate(individual):
    
    total_distance = 0
    total_cost = 0
    #print(individual)
    for car_index, station_index in enumerate(individual):
        car_x, car_y = car_locations[car_index]
        station_x, station_y, station_price = station_locations[station_index]
        distance = math.sqrt((car_x - station_x) ** 2 + (car_y - station_y) ** 2)
        total_distance += distance
        total_cost += station_price
    return total_distance, total_cost

def random_station():
    return random.randint(0, len(station_locations) - 1)

def mutation(individual):
    car_index = random.randint(0, len(individual) - 1)
    individual[car_index] = random_station()
    return individual,None

def crossover(parent1, parent2):
    cxpoint = random.randint(1, len(parent1) - 1)
    child1 = parent1[:cxpoint] + parent2[cxpoint:]
    child2 = parent2[:cxpoint] + parent1[cxpoint:]
    return creator.Individual(child1), creator.Individual(child2)

toolbox.register("evaluate", evaluate)
toolbox.register("individual", tools.initRepeat, creator.Individual, random_station, len(car_locations))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selSPEA2)

# Define the main function
def main():
    population_size = 100
    num_generations = 10
    cxpb = 0.5
    mutpb = 0.2
    # Create the initial population
    population = toolbox.population(n=population_size)

    # Evaluate the fitness of each individual in the population
    fitnesses = [toolbox.evaluate(individual) for individual in population]
    
    for individual, fitness in zip(population, fitnesses):  
        individual.fitness.values = fitness

    # Run the evolution

    for generation in range(num_generations):
        # Select the parents for the next generation
        
        offspring = toolbox.select(population,5# len(population)
                                  )

        # Apply crossover to create the children
        offspring = [toolbox.mate(parent1, parent2) for parent1, parent2 in zip(offspring[::2], offspring[1::2])]
        offspring = [child for pair in offspring for child in pair]

        # Apply mutation to create new individuals
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)

        # Evaluate the fitness of the new individuals
        #print(offspring)
        
        fitnesses = [toolbox.evaluate(individual) for individual in offspring]
        for individual, fitness in zip(offspring, fitnesses):
            individual.fitness.values = fitness

        # Combine the parents and children to create the next generation
        population[:] = toolbox.select(population + offspring, population_size)

        # Print the best individuals in the current generation
        best_individuals = tools.selBest(population, k=10)
        #print("Generation", generation)
        for individual in best_individuals:
            pass
            #print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
        # extract Pareto front
        pareto_front = tools.sortNondominated(population, k=5, first_front_only=True)[0]
    print("Pareto Front:")
    for individual in pareto_front:
        print("  Distance:", individual.fitness.values[0], "Cost:", individual.fitness.values[1], "Assignments:", individual)
 
    print(offspring)

main()

Pareto Front:
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 72.89435788732315 Cost: 15.0 Assignments: [1, 1, 1, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 63.81671736583818 Cost: 25.0 Assignments: [3, 3, 4, 6]
  Distance: 89.86492063580029 Cost: 5.0 Assignments: [6, 6, 1, 6]
  Distance: 89.86492063580029 Cost: 5.0 Assignmen